# Analyse et Prévision de la Scolarisation des Filles en Afrique Subsaharienne

Ce notebook présente une **analyse des données de scolarisation des filles** en Afrique subsaharienne, issues de la **Banque mondiale**. L'objectif principal est de comprendre l'évolution du pourcentage de scolarisation des filles par pays au fil des années et de produire des **prévisions pour les années futures**.

Les étapes réalisées dans ce notebook incluent :  
1. **Chargement et nettoyage des données** : traitement des valeurs manquantes et transformation des données pour l'analyse.  
2. **Analyse exploratoire** : visualisation des tendances historiques, identification des pays avec les meilleures et les moins bonnes performances.  
3. **Prévisions avec Prophet** : modélisation des séries temporelles pour estimer l'évolution future du taux de scolarisation.  
4. **Évaluation des modèles** : comparaison des prévisions avec des méthodes simples comme la moyenne mobile, calcul des métriques MAE et MAPE pour quantifier la précision des prédictions.  
5. **Visualisations** : graphiques continus et informatifs pour mieux comprendre les tendances et les incertitudes des prévisions.

Ce travail vise à fournir des insights utiles pour toute personne intéressée par l’évolution de la scolarisation des filles en Afrique subsaharienne.

Vos retours et suggestions sont les bienvenus afin d'améliorer l'analyse et les prévisions.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Lecture et nettoyage des données
- Chargement du fichier Excel `subsaharan.xlsx`
- Transposition pour mettre les années en lignes
- Suppression de la colonne "Country Name" si présente
- Remise en forme des colonnes en `ds` (date) et `y` (valeur)
- Conversion de `ds` en datetime et nettoyage des valeurs manquantes


In [ ]:
data = pd.read_excel('subsaharan.xlsx').T
data = data.drop("Country Name", errors='ignore') 
data = data.reset_index()  
data.columns = ['ds', 'y']

data['ds'] = pd.to_datetime(data['ds'], format='%Y')
data['y'] = data['y'].astype(float)
data = data.dropna()
data

# Visualisation historique
- Courbe de l'évolution du pourcentage en Afrique subsaharienne
- Chaque point représente une année
- Permet de détecter visuellement les tendances et anomalies


In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(15,6))
sns.lineplot(data=data, x='ds', y='y', marker='o', color='blue', linewidth=2)
plt.title("Pourcentage en Afrique subsaharienne (historique)", fontsize=16)
plt.xlabel("Année")
plt.ylabel("Pourcentage")
plt.xticks(rotation=45)
plt.show()

# Séparation des données en train et test
- Les années jusqu'en 2007 sont utilisées pour entraîner le modèle
- Les années après 2007 sont réservées pour la validation
- Permet d'évaluer la qualité des prédictions de Prophet


In [ ]:
train = data[data['ds'].dt.year <= 2007]
test = data[data['ds'].dt.year > 2007]

# Modélisation avec Prophet
- Création du modèle Prophet avec un intervalle de confiance à 95%
- Entraînement du modèle sur le jeu train
- Préparation d'un DataFrame futur pour prédire sur test + 20 ans


In [ ]:
model = Prophet(interval_width=0.95)
model.fit(train)

future = model.make_future_dataframe(periods=len(test) + 20, freq='Y') 
forecast = model.predict(future)

# Évaluation de la prédiction
- Comparaison entre valeurs réelles (test) et prévisions (yhat)
- Calcul des métriques :
    - MAE : Mean Absolute Error
    - MAPE : Mean Absolute Percentage Error
- Permet de quantifier la précision du modèle


In [ ]:
y_true = test['y'].values
y_pred = forecast['yhat'].iloc[:len(train) + len(test)].tail(len(test)).values

mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f"MAE: {mae:.2f}, MAPE: {mape:.2f}%")

# Visualisation des prédictions
- Affichage combiné :
    - Historique (train)
    - Valeurs réelles (test)
    - Prédictions sur test
    - Prévisions sur 20 ans
- Permet de comparer visuellement la qualité de la prédiction et la tendance future


In [ ]:
plt.figure(figsize=(15,6))
plt.plot(train['ds'], train['y'], label='Historique', color='blue')
plt.plot(test['ds'], y_true, label='Vrai Test', color='green')
plt.plot(test['ds'], y_pred, label='Prédiction', color='red', linestyle='--')
plt.plot(forecast['ds'].iloc[len(train)+len(test):], forecast['yhat'].iloc[len(train)+len(test):], 
         label='Prévision 20 ans', color='orange', linestyle='--')
plt.legend()
plt.xlabel("Année")
plt.ylabel("Pourcentage")
plt.title("Historique, test et prévisions Sub-Saharan")
plt.show()